# CSCI 3202, Spring 2022


# Lecture 19:  Genetic Algorithms Solutions

Before we begin, let's load a few packages that we might find useful.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

<br>


## Maximizing some objective delight

Suppose we are packing a lunch a trying to decide what to bring along.  There are four food options in front of us:
1. an apple,
2. a peanut butter and jelly sandwich,
3. a cookie, and
4. some gravel.

Now we aren't very food-savvy, but we are algorithm-savvy, so we decide to use a ***genetic algorithm*** to decide which of these food items we should pack for lunch.

We can represent the `state` of our lunch as a length 4 bit-string.  For example, `state=[1,1,0,1]` represents a lunch that includes an apple, a PB&J, and some gravel, but no cookie. Yum!

The first step will be to define some objective function to maximize. Let's call this our `delight` with the `state` of our lunch, if we eat all of the items in the lunch.  I can only speak for myself, but the apple will delight me a bit, the PB&J will delight me a little more than the apple, and the cookie will delight me the most of all! But eating that gravel will **not** delight me. In fact, it will reduce my total delight by a substantial amount.

In [2]:
def delight(state):
    goodness = 6            # how happy am I without a lunch?
    goodness += 1*state[0]  # apples are okay...
    goodness += 2*state[1]  # but PB&J is better! (twice as good, apparently)
    goodness += 4*state[2]  # and cookies are even better still!
    goodness += -5*state[3] # and gravel is disgusting.
    return goodness

Let's initialize our genetic algorithm population with each of the four lunches possible by bringing only a single item.

In [3]:
initial_population = [[1,0,0,0],[0,0,0,1],[0,1,0,0],[0,0,1,0]]

... And define our class structure to solve this `problem`:

In [4]:
class problem:
    
    def __init__(self, initial_population, objective_function, mutation_probability, fitness_goal):
        '''
        initial_population = list of lists; each sub-list is a dna string for a population member
        objective_function = objective function to maximize
        mutation_probability = probability that any given child has a mutation
        fitness_goal = fitness goal to achieve (stopping criterion, once member reaches this)
        '''
        self.population = initial_population
        self.initial_population = initial_population
        self.objective_function = objective_function
        self.p_mutate = mutation_probability
        self.n_pop = len(initial_population)
        self.n_dna = len(initial_population[0])
        self.fitness_goal = fitness_goal

    def fitness(self):
        '''
        calculate each population member's probability of being selected for
        reproduction based on performance on objective function
        '''
        performance = []
        for k in range(self.n_pop):
            performance.append(self.objective_function(self.population[k]))
        total = sum(performance)
        p_reproduce = [perf/sum(performance) for perf in performance]
        return p_reproduce
        
    def reproduce(self, parent1, parent2):
        # last DNA snippet from parent1:
        split = np.random.randint(low=1, high=self.n_dna)
        child = parent1[:split] + parent2[split:]
        return child

    def mutate(self, child):
        # which gene to mutate?
        gene = np.random.randint(low=0, high=self.n_dna)
        if child[gene]==0:
            child[gene]=1
        elif child[gene]==1:
            child[gene]=0
        return child
    
    

And lastly, we need to turn our genetic algorithm code into actual code!

In [5]:
def genetic_algorithm(problem, n_iter):
    
    for t in range(n_iter):
        
        new_generation = []
        
        for k in range(problem.n_pop):
            
            # select for reproduction
            p_reproduce = problem.fitness()
            ind_parents = np.random.choice(range(0,problem.n_pop), size=2, p=p_reproduce, replace=False)
            parent1, parent2 = problem.population[ind_parents[0]], problem.population[ind_parents[1]]
            
            # reproduce
            child = problem.reproduce(parent1, parent2)
            
            # mutate
            l_mutate = np.random.choice([True, False], p=[problem.p_mutate, 1-problem.p_mutate])
            if l_mutate:
                child = problem.mutate(child)
            
            # add to new generation
            new_generation.append(child)
        
        # set problem.population = new generation
        problem.population = new_generation
        
        # exit criterion check
        performance = [problem.objective_function(member) for member in problem.population]
        
        best_member = max(zip(performance, problem.population))
        
        if best_member[0] >= problem.fitness_goal:
            return best_member

    print('reached n_iter')

    return False

Now, the moment of truth:  will we eat cookies for lunch?  Or gravel?

In [9]:
genetic_problem = problem(initial_population=initial_population, 
                          fitness_goal=13, 
                          mutation_probability=0.1, 
                          objective_function=delight)
out = genetic_algorithm(genetic_problem, 200)
print(out)

(13, [1, 1, 1, 0])
